<a href="https://colab.research.google.com/github/srujaan/DL-with-PyTorch/blob/master/find_units.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import torch

In [21]:
# "t_c" is temperature in celcius
# "t_u" is temperature are unknown units
t_c = [0.5, 14.0, 15.0, 28.0, 11.0, 8.0, 3.0, -4.0, 6.0, 13.0, 21.0]
t_u = [35.7, 55.9, 58.2, 81.9, 56.3, 48.9, 33.9, 21.8, 48.4, 60.4, 68.4]
t_c = torch.tensor(t_c)
t_u = torch.tensor(t_u)

In [22]:
# model
def model(t_u, w, b):
  return w * t_u + b

In [23]:
# loss function
def loss_fn(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [24]:
# initialize w,b parameteres
w = torch.ones(())
b = torch.ones(())

In [25]:
t_p = model(t_u, w, b)
t_p

tensor([36.7000, 56.9000, 59.2000, 82.9000, 57.3000, 49.9000, 34.9000, 22.8000,
        49.4000, 61.4000, 69.4000])

In [26]:
# check value of loss
loss = loss_fn(t_p, t_c)
loss

tensor(1847.4844)

In [27]:
# Decreasing loss

delta = 0.1

loss_rate_of_change_w = \
  (loss_fn(model(t_u, w + delta, b), t_c) - 
   loss_fn(model(t_u, w - delta, b), t_c)) / (2.0 * delta)

In [28]:
# learning rate should be low as possible

learning_rate = 1e-2
w = w - learning_rate * loss_rate_of_change_w

In [29]:
# we can do same for b

loss_rate_of_change_b = \
  (loss_fn(model(t_u, w, b + delta), t_c) - 
   loss_fn(model(t_u, w, b - delta), t_c)) / (2.0 * delta)

b = b - learning_rate * loss_rate_of_change_b

In [30]:
def dloss_fn(t_p, t_c):
  dsq_diffs = 2 * (t_p - t_c) / t_p.size(0) # The division is from the derivative mean
  return dsq_diffs


In [31]:
# Applying derivatives to the model

def dmodel_dw(t_u, w, b):
  return t_u

def dmodel_db(t_u, w, b):
  return 1.0

In [32]:
# Putting all this together

def grad_fn(t_u, t_c, t_p, w, b):
  dloss_dtp = dloss_fn(t_p, t_c)
  dloss_dw = dloss_dtp * dmodel_dw(t_u, w, b)
  dloss_db = dloss_dtp * dmodel_db(t_u, w, b)
  return torch.stack([dloss_dw.sum(), dloss_db.sum()])

In [39]:
# Training loop

def training_loop(n_epochs, learning_rate, params, t_u, t_c):
  for epoch in range(1, n_epochs + 1):
    w, b = params

    t_p = model(t_u, w, b)  # Forward pass
    loss = loss_fn(t_p, t_c)
    grad = grad_fn(t_u, t_c, t_p, w, b) # Backward pass

    params = params - learning_rate * grad

    print('Epoch %d, Loss %f' % (epoch, float(loss)))
    print('  Params: ', params)
    print('  Grad:', grad)

  return params


In [34]:
training_loop(
    n_epochs = 100,
    learning_rate = 1e-2,
    params = torch.tensor([1.0, 0.0]),
    t_u = t_u,
    t_c = t_c)

Epoch 1, Loss 1763.884766
Epoch 2, Loss 5802484.500000
Epoch 3, Loss 19408029696.000000
Epoch 4, Loss 64915905708032.000000
Epoch 5, Loss 217130525461053440.000000
Epoch 6, Loss 726257583152928129024.000000
Epoch 7, Loss 2429183416467662896627712.000000
Epoch 8, Loss 8125122549611731432050262016.000000
Epoch 9, Loss 27176882120842590626938030653440.000000
Epoch 10, Loss 90901105189019073810297959556841472.000000
Epoch 11, Loss inf
Epoch 12, Loss inf
Epoch 13, Loss inf
Epoch 14, Loss inf
Epoch 15, Loss inf
Epoch 16, Loss inf
Epoch 17, Loss inf
Epoch 18, Loss inf
Epoch 19, Loss inf
Epoch 20, Loss inf
Epoch 21, Loss inf
Epoch 22, Loss inf
Epoch 23, Loss nan
Epoch 24, Loss nan
Epoch 25, Loss nan
Epoch 26, Loss nan
Epoch 27, Loss nan
Epoch 28, Loss nan
Epoch 29, Loss nan
Epoch 30, Loss nan
Epoch 31, Loss nan
Epoch 32, Loss nan
Epoch 33, Loss nan
Epoch 34, Loss nan
Epoch 35, Loss nan
Epoch 36, Loss nan
Epoch 37, Loss nan
Epoch 38, Loss nan
Epoch 39, Loss nan
Epoch 40, Loss nan
Epoch 41, Loss

tensor([nan, nan])

In [40]:
# Decrease the learning rate and try again...
# these are called hyperparameters

training_loop(
    n_epochs = 100,
    learning_rate = 1e-4,
    params = torch.tensor([1.0, 0.0]),
    t_u = t_u,
    t_c = t_c)

Epoch 1, Loss 1763.884766
  Params:  tensor([ 0.5483, -0.0083])
  Grad: tensor([4517.2964,   82.6000])
Epoch 2, Loss 323.090515
  Params:  tensor([ 0.3623, -0.0118])
  Grad: tensor([1859.5493,   35.7843])
Epoch 3, Loss 78.929634
  Params:  tensor([ 0.2858, -0.0135])
  Grad: tensor([765.4666,  16.5122])
Epoch 4, Loss 37.552845
  Params:  tensor([ 0.2543, -0.0143])
  Grad: tensor([315.0790,   8.5787])
Epoch 5, Loss 30.540283
  Params:  tensor([ 0.2413, -0.0149])
  Grad: tensor([129.6733,   5.3127])
Epoch 6, Loss 29.351154
  Params:  tensor([ 0.2360, -0.0153])
  Grad: tensor([53.3495,  3.9682])
Epoch 7, Loss 29.148884
  Params:  tensor([ 0.2338, -0.0156])
  Grad: tensor([21.9304,  3.4148])
Epoch 8, Loss 29.113848
  Params:  tensor([ 0.2329, -0.0159])
  Grad: tensor([8.9964, 3.1869])
Epoch 9, Loss 29.107145
  Params:  tensor([ 0.2325, -0.0162])
  Grad: tensor([3.6721, 3.0930])
Epoch 10, Loss 29.105247
  Params:  tensor([ 0.2324, -0.0166])
  Grad: tensor([1.4803, 3.0544])
Epoch 11, Loss 29.

tensor([ 0.2327, -0.0438])

In [41]:
# Normalize the inputs, so they are in range -1.0 to 1.0

t_un = 0.1 * t_u

# updating it in the training loop
training_loop(
    n_epochs = 100,
    learning_rate = 1e-2, # it is set high and didn't blow up
    params = torch.tensor([1.0, 0.0]),
    t_u = t_un, # updated t_u to our new, rescaled t_un.
    t_c = t_c)

Epoch 1, Loss 80.364342
  Params:  tensor([1.7761, 0.1064])
  Grad: tensor([-77.6140, -10.6400])
Epoch 2, Loss 37.574913
  Params:  tensor([2.0848, 0.1303])
  Grad: tensor([-30.8623,  -2.3864])
Epoch 3, Loss 30.871077
  Params:  tensor([2.2094, 0.1217])
  Grad: tensor([-12.4631,   0.8587])
Epoch 4, Loss 29.756193
  Params:  tensor([2.2616, 0.1004])
  Grad: tensor([-5.2218,  2.1327])
Epoch 5, Loss 29.507153
  Params:  tensor([2.2853, 0.0740])
  Grad: tensor([-2.3715,  2.6310])
Epoch 6, Loss 29.392456
  Params:  tensor([2.2978, 0.0458])
  Grad: tensor([-1.2492,  2.8241])
Epoch 7, Loss 29.298828
  Params:  tensor([2.3059, 0.0168])
  Grad: tensor([-0.8071,  2.8970])
Epoch 8, Loss 29.208717
  Params:  tensor([ 2.3122, -0.0124])
  Grad: tensor([-0.6325,  2.9227])
Epoch 9, Loss 29.119415
  Params:  tensor([ 2.3178, -0.0417])
  Grad: tensor([-0.5633,  2.9298])
Epoch 10, Loss 29.030489
  Params:  tensor([ 2.3232, -0.0710])
  Grad: tensor([-0.5355,  2.9295])
Epoch 11, Loss 28.941877
  Params:  t

tensor([ 2.7553, -2.5162])